In [88]:
%pip install spacy pandas typer


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /nvme/geckos/venvs/jupyterlab-3.12/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [58]:
import sys
import json
import pandas as pd
import spacy
from spacy import displacy 
from spacy.matcher import Matcher
from pathlib import Path
import operator as op

from IPython.display import display, Markdown as Md


In [59]:
files = Path("./output/").glob("*.json")
data = [{**json.load(f.open()), "file": f.name} for f in files]
# display(data)


df = pd.DataFrame(data, columns=["tags", "descrip", "title", "url", "file"])
df

,tags,descrip,title,url,file
0,"[go, golangprojects]",\n\n\nSenior Golang backend developer at Cloud...,Golang / Go Job: Senior Golang backend developer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Golang_backend_develop...
1,"[go, golangprojects]",\n\n\nStaff Software Engineer (Golang) at Stre...,Golang / Go Job: Staff Software Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Staff_Software_Engineer_(Gola...
2,"[go, golangprojects]",\n\n\nSenior Software Engineer – Golang at Clo...,Golang / Go Job: Senior Software Engineer – Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer_–_Go...
3,"[go, golangprojects]",\n\n\nBackend Software Engineer (Golang) at In...,Golang / Go Job: Backend Software Engineer (Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Backend_Software_Engineer_(Go...
4,"[go, golangprojects]","\n\n\nSoftware Engineer, Backend at Glue\n\nRe...","Golang / Go Job: Software Engineer, Backend",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Software_Engineer,_Backend.json"
...,...,...,...,...,...
129,[indeed],"At Coinbase, our mission is to increase econom...","Senior Software Engineer, Backend - Platform (...",https://www.indeed.com/rc/clk?jk=a2dcff952fa26...,"Senior_Software_Engineer,_Backend_-_Platform_(..."
130,[indeed],"\n$115,000 - $375,000 per year",Software Engineer or DevOps Specialist,https://www.indeed.com/rc/clk?jk=1977d16a0dfbe...,Software_Engineer_or_DevOps_Specialist.json
131,[indeed],Profile insightsFind out how your skills align...,Staff Engineer – DevOps (REMOTE),https://www.indeed.com/rc/clk?jk=241294fe2cac5...,Staff_Engineer_–_DevOps_(REMOTE).json
132,[indeed],Profile insightsFind out how your skills align...,Senior VBA Developer,https://www.indeed.com/rc/clk?jk=35706498cac80...,Senior_VBA_Developer.json


In [60]:
df.query("title.str.contains('Golang')")

,tags,descrip,title,url,file
0,"[go, golangprojects]",\n\n\nSenior Golang backend developer at Cloud...,Golang / Go Job: Senior Golang backend developer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Golang_backend_develop...
1,"[go, golangprojects]",\n\n\nStaff Software Engineer (Golang) at Stre...,Golang / Go Job: Staff Software Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Staff_Software_Engineer_(Gola...
2,"[go, golangprojects]",\n\n\nSenior Software Engineer – Golang at Clo...,Golang / Go Job: Senior Software Engineer – Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer_–_Go...
3,"[go, golangprojects]",\n\n\nBackend Software Engineer (Golang) at In...,Golang / Go Job: Backend Software Engineer (Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Backend_Software_Engineer_(Go...
4,"[go, golangprojects]","\n\n\nSoftware Engineer, Backend at Glue\n\nRe...","Golang / Go Job: Software Engineer, Backend",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Software_Engineer,_Backend.json"
5,"[go, golangprojects]",\n\n\nSenior Software Engineer at Posit\n\nRem...,Golang / Go Job: Senior Software Engineer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer.json
6,"[go, golangprojects]",\n\n\nGolang Developer (Ninja Team) at Semrush...,Golang / Go Job: Golang Developer (Ninja Team),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Golang_Developer_(Ninja_Team)...
7,"[go, golangprojects]",\n\n\nGolang Engineer at JUXT\n🇬🇧 United Kingd...,Golang / Go Job: Golang Engineer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Golang_Engineer.json
8,"[go, golangprojects]",\n\n\nPlatform Engineer (Golang) at EDB\n\nRem...,Golang / Go Job: Platform Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Platform_Engineer_(Golang).json
9,"[go, golangprojects]","\n\n\nSenior Engineer, Golang at Fullsteam\n\n...","Golang / Go Job: Senior Engineer, Golang",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Senior_Engineer,_Golang.json"


In [61]:
!{sys.executable} -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /nvme/geckos/venvs/jupyterlab-3.12/bin/python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [62]:
nlp = spacy.load("en_core_web_sm")

In [63]:
location_mat = Matcher(nlp.vocab)
location_pats = [
    [{"ENT_TYPE": "LOC"}],
    [{"ENT_TYPE": "NORP"}],
    [{"ENT_TYPE": "GPE"}],
    [{"ENT_TYPE": "TIME"}],
    [{"LOWER": "worldwide"}],
]
location_mat.add("LOCATION", location_pats)
remote_mat = Matcher(nlp.vocab)
remote_pats = [
    [{"LOWER": "remote"}],
]
remote_mat.add("REMOTE", remote_pats)

In [72]:
def collect_matcher(m, doc, window=0):
    return [doc[s-window:e+window].as_doc() for _, s, e in m(doc)]

def add_remote_metadata(row):
    doc = nlp(row.descrip)
    # Find remote word
    remotes = collect_matcher(remote_mat, doc, 5)
    if remotes:
        locations = set(str(l).strip() for doc in remotes for l in collect_matcher(location_mat, doc))
        if locations:
            return {**row.metadata, "remote": " ".join(sorted(map(str, locations)))}
    return row.metadata

df["metadata"] = [dict() for _ in range(df.index.size)]
df["metadata"] = df.apply(add_remote_metadata, axis=1)
df

,tags,descrip,title,url,file,metadata
0,"[go, golangprojects]",\n\n\nSenior Golang backend developer at Cloud...,Golang / Go Job: Senior Golang backend developer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Golang_backend_develop...,{'remote': 'Americas'}
1,"[go, golangprojects]",\n\n\nStaff Software Engineer (Golang) at Stre...,Golang / Go Job: Staff Software Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Staff_Software_Engineer_(Gola...,{'remote': 'Amsterdam European'}
2,"[go, golangprojects]",\n\n\nSenior Software Engineer – Golang at Clo...,Golang / Go Job: Senior Software Engineer – Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer_–_Go...,{}
3,"[go, golangprojects]",\n\n\nBackend Software Engineer (Golang) at In...,Golang / Go Job: Backend Software Engineer (Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Backend_Software_Engineer_(Go...,{}
4,"[go, golangprojects]","\n\n\nSoftware Engineer, Backend at Glue\n\nRe...","Golang / Go Job: Software Engineer, Backend",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Software_Engineer,_Backend.json",{'remote': 'Americas'}
...,...,...,...,...,...,...
129,[indeed],"At Coinbase, our mission is to increase econom...","Senior Software Engineer, Backend - Platform (...",https://www.indeed.com/rc/clk?jk=a2dcff952fa26...,"Senior_Software_Engineer,_Backend_-_Platform_(...",{}
130,[indeed],"\n$115,000 - $375,000 per year",Software Engineer or DevOps Specialist,https://www.indeed.com/rc/clk?jk=1977d16a0dfbe...,Software_Engineer_or_DevOps_Specialist.json,{}
131,[indeed],Profile insightsFind out how your skills align...,Staff Engineer – DevOps (REMOTE),https://www.indeed.com/rc/clk?jk=241294fe2cac5...,Staff_Engineer_–_DevOps_(REMOTE).json,{}
132,[indeed],Profile insightsFind out how your skills align...,Senior VBA Developer,https://www.indeed.com/rc/clk?jk=35706498cac80...,Senior_VBA_Developer.json,{}


In [73]:
!mkdir output2/ || :

mkdir: cannot create directory ‘output2/’: File exists


In [74]:
for _, row in df.iterrows():
    with open(f"output/{row.file}", "w") as f:
        f.write(json.dumps({ "title": row.title,
                              "descrip": row.descrip,
                            "url": row.url,
                            "tags": row.tags,
                            "metadata": row.metadata}))
        print(f"File {f.name} written")

File output2/Golang___Go_Job:_Senior_Golang_backend_developer.json written
File output2/Golang___Go_Job:_Staff_Software_Engineer_(Golang).json written
File output2/Golang___Go_Job:_Senior_Software_Engineer_–_Golang.json written
File output2/Golang___Go_Job:_Backend_Software_Engineer_(Golang).json written
File output2/Golang___Go_Job:_Software_Engineer,_Backend.json written
File output2/Golang___Go_Job:_Senior_Software_Engineer.json written
File output2/Golang___Go_Job:_Golang_Developer_(Ninja_Team).json written
File output2/Golang___Go_Job:_Golang_Engineer.json written
File output2/Golang___Go_Job:_Platform_Engineer_(Golang).json written
File output2/Golang___Go_Job:_Senior_Engineer,_Golang.json written
File output2/Golang___Go_Job:_Senior_Golang_Developer.json written
File output2/Golang___Go_Job:_Senior_Developer,_Back-End,_Golang_(Greece).json written
File output2/Senior_Rust_Engineer.json written
File output2/Web3_Backend_Engineer_(Rust).json written
File output2/Rust_Engineer_-_Ge